## Texas Water Scraper
* Get a list of sites for the year to scrape
* Loop through each size; generate and pull a report
* Scrape the report details into a csv file
---
Requires:
* The selenium package and the geckodriver.exe file (for Mozilla browsers).

In [21]:
#Import libraries
import os, glob, time
import pandas as pd
import requests
import unicodedata
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [22]:
#Settings
year = 2016
theURL = 'http://www2.twdb.texas.gov/ReportServerExt/Pages/ReportViewer.aspx?%2fWU%2fSumFinal_WUG_Entity_Detail_2016&rs:Command=Render'

### 1. Scrape a list of utilities from the base URL

In [23]:
#Call the page and convert it to 'soup'
response = requests.get(theURL)
soup = BeautifulSoup(response.content,'lxml')

#Extract the control from the 'soup'
tbl = soup.find(id="ReportViewerControl_ctl04_ctl03")

#Create a list of utilities and populate values from each option in the control
utilities = []
values = []
for option in tbl.findAll('option')[1:]: #Skips the first item
    #Remove unicode chars
    utility = unicodedata.normalize("NFKD",option.text)
    utilities.append(utility)
    values.append(option.attrs['value'])
    


### 2. Loop through each utility and extract its report
This step requires the selenium package to open a remotely controlled browser, mimic selecting the utility and clicking the View Report button, then waiting for the report to complete and appear. Once that occurs the contents stored as a local HTML file for later scraping. 

In [32]:
#Create the selenium browser object
browser = webdriver.Firefox(executable_path='./geckodriver.exe')

#Open the page in the selenium browser
browser.get(theURL)

#Loop through each utility
for value,utility in zip(values,utilities):
        
    #Set the output filename from the utility and year values
    outFN = "./Pages/{}_{}.html".format(utility.replace(" ","_"),year)
    
    #Skip if already downloaded
    if os.path.exists(outFN):
        #print("Already processed.")
        continue

    #Status
    print("{}:\t".format(utility),end="")


    #Select the item and 'click' it
    utilControl = Select(browser.find_element_by_id('ReportViewerControl_ctl04_ctl03_ddValue'))
    utilControl.select_by_value(value)

    #Select the View Report button to retrieve the report
    browser.find_element_by_xpath('//*[@id="ReportViewerControl_ctl04_ctl00"]').click()

    #Wait for the report to complete (i.e. when the utility's name appears on the active page)
    searchText = "{}</div>".format(utility)

    print("  fetching report",end="... ")
    while True:
        pageSource = browser.page_source
        elapsed_time = time.time() - start_time
        #Wait until the "cancel" <div> object disappears:
        if not browser.find_element_by_id('ReportViewerControl_AsyncWait_Wait').is_displayed():
            print("Complete! Saved to {}".format(outFN))
            with open(outFN,'w') as outfile:
                outfile.write(browser.page_source)
                #---------------------------------Check for 2nd page
                soup = BeautifulSoup(pageSource,'lxml') 
                pageCtrl = soup.find(id='ReportViewerControl_ctl05_ctl00_TotalPages')
                if '2' in pageCtrl.contents[0]: # There's a 2nd page
                    print("   fetching 2nd page",end="... ")
                    #Click to fetch the 2nd page
                    browser.find_element_by_xpath('//*[@id="ReportViewerControl_ctl05_ctl00_Next_ctl00_ctl00"]').click()
                    #Wait until page loads
                    while True:
                        page2Source = browser.page_source
                        ctrl = BeautifulSoup(page2Source,'lxml').find(id='ReportViewerControl_ctl05_ctl00_TotalPages')
                        if "?" not in ctrl.contents[0]:
                            outFN2 = outFN[:-5]+"_pg2.html"
                            with open(outFN2,'w') as outFile2: 
                                outFile2.write(browser.page_source)
                            break
                #---------------------------------
            break

    #Save the page to a file
    #with open(outFN,'w') as outfile:
    #    outfile.write(browser.page_source)


AUBREY:	  fetching report... Complete! Saved to ./Pages/AUBREY_2016.html
AUSTIN:	  fetching report... Complete! Saved to ./Pages/AUSTIN_2016.html
AUSTIN COUNTY WSC:	

StaleElementReferenceException: Message: The element reference of <option> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


WebDriverException: Message: Failed to decode response from marionette


### 3. With pages downloaded, scrape the data within each into a single dataframe
* Read all html files in the folder
* Open content as soup
* 

In [23]:
htmlFiles = glob.glob(".\\Pages\\*.html")

In [24]:
def getData(df,util,yr):
    tbl = df.iloc[3:]
    tbl.columns = df.iloc[2]
    tbl.insert(0,'Src_Utility',util)
    tbl.insert(1,'Src_Year',yr)
    return tbl

In [53]:
for htmlFile in htmlFiles:
    #Get utility and name from the file
    utility = htmlFile.split("\\")[-1][:-10]
    year = htmlFile[-9:-5]
    print("{},{}".format(utility,year),end="\t")

    #Read the contents
    content = open(htmlFile,'r').read()
    soup = BeautifulSoup(content,'lxml') 

    #Parse 
    bigDiv = soup.find(id='VisibleReportContentReportViewerControl_ctl09')
    tbls = bigDiv.contents[0].contents[0].contents[0].contents[0].contents[0].contents[0].contents[0]

    #Convert results to dataframes
    dfs = pd.read_html(str(tbls))

    #Pull data from tables
    Table1 = getData(dfs[2],utility,year)
    Table2 = getData(dfs[3],utility,year)
    Table3 = getData(dfs[4],utility,year)
    Table4 = getData(dfs[5],utility,year)
    #Table5 = getData(dfs[6],utility,year)

    #Merge data
    if htmlFile == htmlFiles[0]:
        df1 = Table1
        df2 = Table2
        df3 = Table3
        df4 = Table4
        #df5 = Table5
    else:
        df1 = pd.concat((df1,Table1))
        df2 = pd.concat((df2,Table2))
        df3 = pd.concat((df3,Table3))
        df4 = pd.concat((df4,Table4))
        #df5 = pd.concat((df5,Table5))
    
#Write tables out
df1.to_csv("Table1.csv",index=False)
df2.to_csv("Table2.csv",index=False)
df3.to_csv("Table3.csv",index=False)
df4.to_csv("Table4.csv",index=False)
#df5.to_csv("Table5.csv",index=False)

410_WSC,2016	439_WSC,2016	ABERNATHY,2016	ABILENE,2016	ABLES_SPRINGS_WSC,2016	ACTON_MUD,2016	ADDISON,2016	AFTON_GROVE_WSC,2016	AGUA_SUD,2016	AIRLINE_MOBILE_HOME_PARK_LTD,2016	AIR_FORCE_VILLAGE_II_INC,2016	ALAMO,2016	ALAMO_HEIGHTS,2016	ALBANY,2016	ALEDO,2016	ALGONQUIN_WATER_RESOURCES_OF_TEXAS,2016	ALICE,2016	ALLEN,2016	ALPINE,2016	ALTO,2016	ALTO_RURAL_WSC,2016	ALVARADO,2016	ALVIN,2016	ALVORD,2016	AMARILLO,2016	AMHERST,2016	ANAHUAC,2016	ANDERSON_COUNTY_CEDAR_CREEK_WSC,2016	ANDREWS,2016	ANGELINA_WSC,2016	ANGLETON,2016	ANNA,2016	ANNETTA,2016	ANSON,2016	ANTHONY,2016	ANTON,2016	APPLEBY_WSC,2016	AQUA_WSC,2016	ARANSAS_PASS,2016	ARCHER_CITY,2016	ARCHER_COUNTY_MUD_1,2016	ARGYLE_WSC,2016	ARLEDGE_RIDGE_WSC,2016	ARLINGTON,2016	ARMSTRONG_WSC,2016	ARP,2016	ASHERTON,2016	ASPERMONT,2016	ATASCOSA_RURAL_WSC,2016	ATHENS,2016	ATLANTA,2016	AUBREY,2016	AUSTIN,2016	AUSTIN_COUNTY_WSC,2016	AVALON_WATER_SUPPLY_&_SEWER_SERVICE,2016	AXTELL_WSC,2016	AZLE,2016	BACLIFF_MUD,2016	BAFFIN_BAY_WSC,2016	BAIRD,2016	BAKER_ROA

FERN_BLUFF_MUD,2016	FERRIS,2016	FILES_VALLEY_WSC,2016	FIRST_COLONY_MUD_9,2016	FIVE_WAY_WSC,2016	FLATONIA,2016	FLAT_FORK_WSC,2016	FLAT_WSC,2016	FLORENCE,2016	FLORESVILLE,2016	FLOWER_MOUND,2016	FLOYDADA,2016	FLO_COMMUNITY_WSC,2016	FOLLETT,2016	FOREST_HILLS_MUD,2016	FOREST_HILL,2016	FORNEY,2016	FORNEY_LAKE_WSC,2016	FORT_BELKNAP_WSC,2016	FORT_BEND_COUNTY_FWSD_1,2016	FORT_BEND_COUNTY_FWSD_2,2016	FORT_BEND_COUNTY_MUD_115,2016	FORT_BEND_COUNTY_MUD_116,2016	FORT_BEND_COUNTY_MUD_121,2016	FORT_BEND_COUNTY_MUD_128,2016	FORT_BEND_COUNTY_MUD_129,2016	FORT_BEND_COUNTY_MUD_140,2016	FORT_BEND_COUNTY_MUD_149,2016	FORT_BEND_COUNTY_MUD_152,2016	FORT_BEND_COUNTY_MUD_155,2016	FORT_BEND_COUNTY_MUD_158,2016	FORT_BEND_COUNTY_MUD_162,2016	FORT_BEND_COUNTY_MUD_187,2016	FORT_BEND_COUNTY_MUD_23,2016	FORT_BEND_COUNTY_MUD_24,2016	FORT_BEND_COUNTY_MUD_25,2016	FORT_BEND_COUNTY_MUD_26,2016	FORT_BEND_COUNTY_MUD_42,2016	FORT_BEND_COUNTY_MUD_46,2016	FORT_BEND_COUNTY_MUD_47,2016	FORT_BEND_COUNTY_MUD_48,2016	FORT_BEND_COUN

IndexError: list index out of range

In [54]:
df1.to_csv("Table1.csv",index=False)
df2.to_csv("Table2.csv",index=False)
df3.to_csv("Table3.csv",index=False)
df4.to_csv("Table4.csv",index=False)

In [42]:
dfDict = {}
for dfX in dfs:
    if dfX.shape[1] < 2: continue
    if dfX.iloc[1,0] == 'Table 1. Calculated GPCD of the WUG Utility': dfDict['1']=dfX
    if dfX.iloc[1,0] == 'Table 2. Water System(s) of the WUG Utility': dfDict['2']=dfX
    if dfX.iloc[1,0] == 'Table 3. Intake Volumes of Water Systems of the WUG Utility': dfDict['3']=dfX
    if dfX.iloc[1,0] == 'Table 4. Sales Volumes of Water Systems of the WUG Utility': dfDict['4']=dfX  

In [52]:
dfs[6].iloc[1,1]

IndexError: list index out of range